# Formatação dos dados para Regionalização de Bacias usando Redes Neurais

## Importing Packages

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")
sys.path.append("../database")

In [2]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import os

In [3]:
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')

# # Plotting
# import plotly
# import plotly.graph_objs as go
# # import plotly.io as pio
# #import cufflinks as cf
# plotly.offline.init_notebook_mode(connected=True)


## Importing Data

In [4]:
caracFisicas = pd.read_excel("../database/dados.xls",sheet_name="Resultados")
caracFisicas = caracFisicas.drop(["SAT","PES","Nash","Nash Ver"],axis=1)

db = pd.ExcelFile("../database/dados.xls")
basins = db.sheet_names[6:]

In [5]:
caracUtilizadas = caracFisicas.drop(["Precipitação média - P (mm)","Capacidade de armazenamento do solo CAD (mm)",
                                    "Índice de compacidade da bacia - Kc",],axis=1)
caracUtilizadas.head()

,Estações ANA,Declividade - D (%),Comprimento do rio principal - CT (km),Área de contribuição da estação fluviométrica - A (km2),Perímetro da área de contribuição da estação fluviométrica - P (km),Comprimento total de drenagem - CTD (km),Densidade de drenagem - DD (km-1),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr
0,34730000,0.069435,64319.016213,897.371509,211.282396,528.790311,0.589266,56.773914,0.000000
1,34740000,0.064875,108123.318373,2221.988771,181.354250,356.001008,0.160217,59.941362,0.000014
2,34750000,0.056337,328854.100000,19185.920000,1256.998009,10051.126447,0.523880,73.945379,0.588056
3,35050000,0.092084,66700.567865,997.264362,218.297537,606.451358,0.608115,82.375460,0.534352
4,35125000,0.081096,96187.532702,1501.236654,261.225198,1501.190430,0.999969,84.314697,0.920865


In [6]:
caracFFN1 = caracUtilizadas.drop('Estações ANA',axis=1).columns
caracFFN2 = caracFFN1[[2,4,6,7]]
caracFFN3 = caracFFN2

## Formatting Data

In [7]:
def ShiftAndConcat(df,col,n):
    for i in range(n):
        if i ==0:
            temp = df[col]
        else:
            temp = pd.concat([temp,df[col].shift(-i)],axis=1)
            
    temp.columns = [col+"_%d" % (n-1-j) for j in range(n)]
    return temp


In [8]:
# PEQbasin = db.parse(basins[0])
# PEQbasin = PEQbasin.set_index("Data")
# Q = ShiftAndConcat(PEQbasin,"Q",3)
# P = ShiftAndConcat(PEQbasin,"P",3)
# E = PEQbasin["E"]
# dfBasin = pd.concat([P,Q,E],axis=1)
# colNames = dfBasin.columns
# for col in colNames:
#     dfBasin = dfBasin.query("%s != -999" % col)
# dfBasin.reset_index().drop("Data",axis=1)
# E = PEQbasin["E"]
# dfBasin = pd.concat([E,P,Q],axis=1)
# colNames = dfBasin.columns
# for col in colNames:
#     dfBasin = dfBasin.query("%s != -999" % col)
# dfBasin.reset_index(drop=True)
# caracBasin = caracUtilizadas[caracUtilizadas["Estações ANA"] == int(basins[0])]
# caracBasinMatrix = np.ones((dfBasin.shape[0],caracBasin.shape[1])) * caracBasin.values
# caracBasinDf = pd.DataFrame(caracBasinMatrix,columns=caracBasin.columns)
# dfFormatted = pd.concat([caracBasinDf,dfBasin.reset_index().drop("Data",axis=1)],axis=1)
# dfFormatted["Estações ANA"] = dfFormatted["Estações ANA"].astype(int).astype(str)

In [9]:
for k in range(len(basins)):
    PEQbasin = db.parse(basins[k])
    PEQbasin = PEQbasin.set_index("Data")
    Q = ShiftAndConcat(PEQbasin,"Q",3)
    P = ShiftAndConcat(PEQbasin,"P",3)
    E = ShiftAndConcat(PEQbasin,"E",3)
    dfBasin = pd.concat([E,P,Q],axis=1)
    colNames = dfBasin.columns
    for col in colNames:
        dfBasin = dfBasin.query("%s != -999" % col)
    dfBasin
    caracBasin = caracUtilizadas[caracUtilizadas["Estações ANA"] == int(basins[k])]
    caracBasinMatrix = np.ones((dfBasin.shape[0],caracBasin.shape[1])) * caracBasin.values
    caracBasinDf = pd.DataFrame(caracBasinMatrix,columns=caracBasin.columns)
    dfFormatted = pd.concat([caracBasinDf,dfBasin.reset_index().drop("Data",axis=1)],axis=1)
    if k == 0:
        dfCompleted = dfFormatted
    else:
        dfCompleted = pd.concat([dfCompleted, dfFormatted], axis=0)
dfCompleted = dfCompleted.loc[dfCompleted.Q_0.notnull()]

In [10]:

dfCompleted["Estações ANA"] = dfCompleted["Estações ANA"].astype(int).astype(str)

In [11]:
caracDiffs = list(set(caracFFN1).difference(set(caracFFN2))) + ["E_2","E_1"]
caracDiffs2 = caracDiffs + ["Q_2","Q_1"]

In [12]:
dfCompleted = dfCompleted.reset_index(drop=True)#.drop(caracDiffs,axis=1)


In [13]:
maxs = dfCompleted.iloc[:,1:-1].max().values
mins = dfCompleted.iloc[:,1:-1].min().values

In [14]:
dfCompletedNorm = dfCompleted.copy()
dfCompletedNorm.iloc[:,1:-1] = dfCompleted.iloc[:,1:-1].apply(lambda x: (x-mins)/(maxs-mins),axis=1)


## Saving Data

In [15]:
dfCompletedNorm.to_csv("../database/DadosFFN1.csv",index=False)
dfCompletedNorm.drop(caracDiffs,axis=1).to_csv("../database/DadosFFN2.csv",index=False)
dfCompletedNorm.drop(caracDiffs2,axis=1).to_csv("../database/DadosFFN3.csv",index=False)